In [1]:
def get_normalized_receipt (receipt_line):
    assert type (receipt_line) is str
    receipt_item_list = receipt_line.split("\n")
    return receipt_item_list
    raise NotImplementedError()

# Demo:
## print ("First five words:\n{}".format (get_normalized_words (latin_text)[:5]))

In [2]:
def make_itemsets(words):
#    return [set(word) for word in words]
#    set_of_items = []
#    print(set_of_items)
#    for each_string in words:
#        set_of_items.append(set(each_string))
    return words.split(",")

In [3]:
from collections import defaultdict
from itertools import combinations # Hint!
## pair_counts = defaultdict(int)
def update_pair_counts (pair_counts, itemset):
    """
    Updates a dictionary of pair counts for
    all pairs of items in a given itemset.
    """
    for a,b in combinations(itemset,2):
        pair_counts[a,b] +=1
        pair_counts[b,a] +=1
    return(pair_counts)

In [4]:
## item_counts = defaultdict(int)
def update_item_counts(item_counts, itemset):
    for c in itemset:
        item_counts[c] +=1
    return(item_counts)
## itemset_1 = set("error")
## itemset_2 = set("dolor")
#print (update_item_counts(item_counts, itemset_1))
## update_item_counts(item_counts, itemset_1)
## item_counts_table = update_item_counts(item_counts, itemset_2)
## print (item_counts_table)

In [5]:
#threshold = 0.0
#rules={}
def filter_rules_by_conf (pair_counts, item_counts, threshold):
#    rules = {} # (item_a, item_b) -> conf (item_a => item_b)
    # YOUR CODE HERE
    # for each key (x,y) from pairs table and get the value(m)
    # lookup x in items table an get the value (n)
    # threshold = m/n
    # rules table has list of pairs x,y and threshold (m/n)
    for a,b in pair_counts:
        threshold_calc = pair_counts[a,b]/item_counts[a]
#        print(a,b, threshold_calc)
        if (threshold_calc > threshold) and (item_counts[a]>1) :
#            print(a,b, threshold_calc)
            rules [a,b]=threshold_calc
#    raise NotImplementedError()
    return rules
## rules = filter_rules_by_conf(pair_counts_table, item_counts_table, 0.5)
## print (rules)

In [6]:
def gen_rule_str(a, b, val=None, val_fmt='{:.3f}', sep=" = "):
    text = "{} => {}".format(a, b)
    if val:
        text = "conf(" + text + ")"
        text += sep + val_fmt.format(val)
    return text

def print_rules(rules):
    if type(rules) is dict or type(rules) is defaultdict:
        from operator import itemgetter
        ordered_rules = sorted(rules.items(), key=itemgetter(1), reverse=True)
    else: # Assume rules is iterable
        ordered_rules = [((a, b), None) for a, b in rules]
    for (a, b), conf_ab in ordered_rules:
        print(gen_rule_str(a, b, conf_ab))

# Demo:
## print_rules(rules)

In [7]:
def find_assoc_rules(receipts, threshold):
    pair_counts = defaultdict(int)
    item_counts = defaultdict(int)
    all_receipts = get_normalized_receipt(groceries_file)
    for receipt_line in all_receipts:
#        print("each receipt",receipt_line)
        each_item_set = make_itemsets(receipt_line)
#        print("split receipt",each_item_set)
#        print("pair counts before", pair_counts)
        update_pair_counts(pair_counts,each_item_set)
#        print("pair counts after", pair_counts)
#        print("item counts before", item_counts)
        update_item_counts(item_counts,each_item_set)
#        print("item counts after", item_counts)
#        input("Press Enter to continue...")
    filter_rules_by_conf (pair_counts,item_counts, threshold)
    return (rules)

In [8]:
import requests
response = requests.get ('https://raw.githubusercontent.com/cse6040/labs-fa17/master/lab2-assoc_rule_mining/groceries.csv')
groceries_file = response.text  # or response.content for raw bytes
print (groceries_file[0:250] + "...\n... (etc.) ...") # Prints the first 250 characters only

citrus fruit,semi-finished bread,margarine,ready soups
tropical fruit,yogurt,coffee
whole milk
pip fruit,yogurt,cream cheese ,meat spreads
other vegetables,whole milk,condensed milk,long life bakery product
whole milk,butter,yogurt,rice,abrasive clea...
... (etc.) ...


In [9]:
rules={}
all_receipts = get_normalized_receipt(groceries_file)
#for receipt_line in all_receipts:
#    print("each line",receipt_line)
#    set_of_items_on_receipt = make_itemsets(receipt_line)
#    print("individual",set_of_items_on_receipt)
groceries_rules=find_assoc_rules(all_receipts,0.6)
print_rules (groceries_rules)

conf(preservation products => citrus fruit) = 1.000
conf(preservation products => whipped/sour cream) = 1.000
conf(kitchen utensil => whole milk) = 0.750
conf(honey => whole milk) = 0.733
conf(frozen fruits => other vegetables) = 0.667
conf(cereals => whole milk) = 0.643
conf(salad dressing => other vegetables) = 0.625
conf(rice => whole milk) = 0.613
